## Y Encode and Decode

In [1]:
import cv2
import numpy as np

import cv2
import numpy as np
import pandas as pd
from matplotlib import cm
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from matplotlib.colors import Normalize


# write arr to excel
def write_xlsx(arr, path):
    return pd.DataFrame(arr).to_excel(path)

# load img as arr
def load_img(img):
    return cv2.imread(img)

# convert rgb to yub
def rgb2yuv(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

def bgr2rgb(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Split channel
def split_channel(img):
    return cv2.split(img)

# Get scale alpha
def get_Q(q_jpeg):
    QTable = np.array([[16,11,10,16, 24,40, 51, 61],
                   [12,12,14,19, 26,48, 60, 55],
                   [14,13,16,24, 40,57, 69, 56],
                   [14,17,22,29, 51,87, 80, 62],
                   [18,22,37,56, 68,109,103,77],
                   [24,35,55,64, 81,104,113,92],
                   [49,64,78,87,103,121,120,101],
                   [72,92,95,98,112,100,103,99]])
    
    if q_jpeg < 50 and q_jpeg > 1:
        alpha = np.floor(5000.0/q_jpeg)
        Q = QTable*alpha
    elif q_jpeg < 100:
        alpha = np.floor(200 - 2*q_jpeg)
        Q = QTable*alpha
    elif q_jpeg == 100:
        Q = np.ones((8, 8))
    else:
        raise AttributeError("Quality Factor must be in the range [1..100]")
    return Q




In [2]:
# Load source.jpg
source = load_img("./input/source.jpg")
# convert source.jpg to YUV Channel
yuv = rgb2yuv(source)
# split YUV channel
y, u, v = split_channel(yuv)

In [ ]:
# Perform DCT with Y Channel as source
np.set_printoptions(precision=4, suppress=True)
source = y

def demo(qf, source):
    QF = qf

    # Init block 8x8
    block = 8
    h, w = np.array(source.shape[:2])/block*block
    h, w = int(h), int(w)
    source = source[:h, :w]

    blocksV = int(h/block)
    blocksH = int(w/block)
    vis0 = np.zeros((h, w), np.float32)
    result = np.zeros((h, w), np.float32)
    vis0[:h, :w] = source

    Q = get_Q(QF)
    print('q_JPEG: ', QF)
    print('\nmatrix Q_Y*alpha:\n', Q)

    # Perform DCT 8x8
    for row in range(blocksV):
        for col in range(blocksH):
            current_block = vis0[row*block:(row+1)*block, col*block:(col+1)*block]
            dct = cv2.dct(current_block)
            if row==0 and col==0:
                print('\nfirst block 8x8 of DCT(source) lossless:\n', np.round(dct).astype(int))
            result_ = np.round(dct/Q).astype(int)
            result[row*block:(row+1)*block, col*block:(col+1)*block] = result_

    # Write source, and result DCT as result_DCT.jpg
    cv2.imwrite("./output/result_DCT_{}.jpg".format(QF), np.hstack([y, result]))
    # Write array of result DCT to excel
    write_xlsx(result, "./output/result_DCT_{}.xlsx".format(QF))

    # Convert back use IDCT
    back = np.zeros((h, w), np.float32)
    for row in range(blocksV):
        for col in range(blocksH):
            current_block = result[row*block:(row+1)*block, col*block:(col+1)*block]
            idct = cv2.idct(current_block*Q)
            back[row*block:(row+1)*block, col*block:(col+1)*block] = np.round(idct).astype(int)
            if row==0 and col==0:
                print('\nfirst block 8x8 of IDCT(source) lossy:\n', np.round(idct).astype(int))

    cv2.imwrite("./output/result_IDCT_{}.jpg".format(QF), np.hstack([y, result, back]))
    print('\nOpen file : ./output/result_IDCT_{}.jpg\n'.format(QF))
    
QFs = [100.0, 99.0, 98.0, 96.0, 94.0, 92.0]

for qf in QFs:
    print('For QF : ', qf)
    demo(qf, source)
    print('=============================\n\n\n\n\n')

For QF :  100.0
q_JPEG:  100.0

matrix Q_Y*alpha:
 [[1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1.]]

first block 8x8 of DCT(source) lossless:
 [[578  -6   6  -4   3  -1   1  -2]
 [ -3   8  -2   0  -1   0  -1   0]
 [ -1  -1   2   1  -1   1   0   0]
 [  0   1  -2   0   0   0   0   0]
 [  1  -1   1   1   0   0   0   0]
 [  0   1  -1   0   0   0   0   0]
 [  1  -1   0   1   0   0   1   0]
 [ -1   0   0   0   0   0   0   0]]

first block 8x8 of IDCT(source) lossy:
 [[73 73 72 72 70 71 71 72]
 [72 73 72 72 70 70 71 73]
 [72 72 72 72 71 71 71 74]
 [72 73 72 73 72 72 72 75]
 [71 72 71 72 72 72 72 76]
 [71 71 72 73 72 73 73 77]
 [71 71 71 72 71 72 73 77]
 [72 70 70 70 71 74 76 80]]

Open file : ./output/result_IDCT_100.0.jpg






For QF :  99.0
q_JPEG:  99.0

matrix Q_Y*alpha:
 [[ 32.  22.  20.  32.  48.  80. 102. 122.]
 [ 24.  

## RESULT for QF=100.0
![RESULT](./output/result_IDCT_100.0.jpg)